In [1]:
"""
#  Pinecone 벡터 DB 업로드 (Upstage Embedding)

"""
##  1단계: 환경 설정 및 라이브러리 임포트
# 패키지 설치 (최초 1회)
# !pip install python-dotenv pandas langchain langchain-upstage langchain-pinecone pinecone-client tqdm

'\n#  Pinecone 벡터 DB 업로드 (Upstage Embedding)\n\n'

In [2]:

import os
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

# LangChain
from langchain_core.documents import Document
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

print("✅ 라이브러리 임포트 완료")

✅ 라이브러리 임포트 완료


In [3]:

load_dotenv()

True

In [4]:
# =========================
# Path 설정
# =========================
BASE_DIR = Path.cwd().parents[1]
CSV_DIR = BASE_DIR / "data" / "processed" / "csv"

LAW_CSV = CSV_DIR / "law.csv"
RULE_CSV = CSV_DIR / "rule.csv"
CASE_CSV = CSV_DIR / "case.csv"

In [5]:
# .env 파일 로드
env_path = BASE_DIR / ".env"
load_dotenv(env_path)

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
INDEX_NAME = "real-estate-law-index"

# API 키 확인
print("🔑 API 키 확인:")
print(f"  PINECONE_API_KEY: {'✅' if PINECONE_API_KEY else '❌ 없음'}")
print(f"  UPSTAGE_API_KEY: {'✅' if UPSTAGE_API_KEY else '❌ 없음'}")
print(f"  OPENAI_API_KEY: {'✅' if OPENAI_API_KEY else '❌ 없음'}")
print(f"\n📌 Pinecone 인덱스명: {INDEX_NAME}")

🔑 API 키 확인:
  PINECONE_API_KEY: ✅
  UPSTAGE_API_KEY: ✅
  OPENAI_API_KEY: ✅

📌 Pinecone 인덱스명: real-estate-law-index


In [6]:
##  3단계: CSV 파일 로드 및 통합

In [7]:
print(BASE_DIR)

c:\project-AI2\실프로젝트


In [8]:
def load_all_csv():
    """law + rule + case CSV 모두 로드 및 통합"""
    dfs = []
    
    if LAW_CSV.exists():
        df_law = pd.read_csv(LAW_CSV)
        print(f"✅ law.csv 로드: {len(df_law)} rows")
        dfs.append(df_law)
    
    if RULE_CSV.exists():
        df_rule = pd.read_csv(RULE_CSV)
        print(f"✅ rule.csv 로드: {len(df_rule)} rows")
        dfs.append(df_rule)
    
    if CASE_CSV.exists():
        df_case = pd.read_csv(CASE_CSV)
        print(f"✅ case.csv 로드: {len(df_case)} rows")
        dfs.append(df_case)
    
    if not dfs:
        raise FileNotFoundError("CSV 파일이 없습니다. 먼저 parse_all.py를 실행하세요.")
    
    # 모두 합치기
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = df_all.fillna("")
    print(f"\n📊 전체 데이터: {len(df_all)} rows")
    
    return df_all

# 실행
df = load_all_csv()

✅ law.csv 로드: 145 rows
✅ rule.csv 로드: 18 rows
✅ case.csv 로드: 27 rows

📊 전체 데이터: 190 rows


In [9]:
# 데이터 미리보기
print("📋 데이터 샘플:")
df.head()

📋 데이터 샘플:


,id,content,law_family,law_name,law_type,article,article_title,priority,effective_date,source_file,parsed_at,case_number,keywords
0,2530eb03-cc6c-4ddf-8747-7d204c1832ac,채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 ...,주택임대차,민법,일반법,제390조,채무불이행과 손해배상,4,2026-01-02,민법(법률)(제20432호)(20260101)-간략.docx,2026-01-16T15:39:53.494533,,
1,3b84d559-0a0e-4a92-aeba-5690447f70c0,차주가 차용물을 반환하는 때에는 이를 원상에 회복하여야 한다. 이에 부속시킨 물건은...,주택임대차,민법,일반법,제615조,차주의 원상회복의무와 철거권,4,2026-01-02,민법(법률)(제20432호)(20260101)-간략.docx,2026-01-16T15:39:53.494533,,
2,30f0bf1b-3801-4d21-b17e-3d5c560828f2,"임대차는 당사자 일방이 상대방에게 목적물을 사용, 수익하게 할 것을 약정하고 상대방...",주택임대차,민법,일반법,제618조,임대차의 의의,4,2026-01-02,민법(법률)(제20432호)(20260101)-간략.docx,2026-01-16T15:39:53.494533,,
3,0652ba8d-3dc8-4f78-96ae-95f9b58626a1,처분의 능력 또는 권한없는 자가 임대차를 하는 경우에는 그 임대차는 다음 각호의 기...,주택임대차,민법,일반법,제619조,"처분능력, 권한없는 자의 할 수 있는 단기임대차",4,2026-01-02,민법(법률)(제20432호)(20260101)-간략.docx,2026-01-16T15:39:53.494533,,
4,43e24370-bc62-45a2-a967-2233973925e3,"전조의 기간은 갱신할 수 있다. 그러나 그 기간만료전 토지에 대하여는 1년, 건물 ...",주택임대차,민법,일반법,제620조,단기임대차의 갱신,4,2026-01-02,민법(법률)(제20432호)(20260101)-간략.docx,2026-01-16T15:39:53.494533,,


In [10]:
# Priority 통계
print("📊 우선순위별 통계:")
priority_stats = df.groupby(['priority', 'law_type']).size().reset_index(name='count')
priority_stats

📊 우선순위별 통계:


,priority,law_type,count
0,1,특별법,47
1,2,시행령,40
2,3,대법원규칙,18
3,3,시행규칙,12
4,4,일반법,46
5,7,상담사례,27


In [11]:
##  4단계: DataFrame → LangChain Document 변환

In [12]:
def csv_to_documents(df):
    """
    DataFrame → LangChain Document 변환
    metadata에 모든 컬럼 포함 (priority 포함!)
    """
    documents = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Document 변환 중"):
        # content를 page_content로
        content = row.get("content", "")
        
        # 나머지는 모두 metadata로
        metadata = {
            "id": row.get("id", ""),
            "law_family": row.get("law_family", ""),
            "law_name": row.get("law_name", ""),
            "law_type": row.get("law_type", ""),
            "article": row.get("article", ""),
            "article_title": row.get("article_title", ""),
            "priority": int(row.get("priority", 99)),  # 중요!
            "effective_date": row.get("effective_date", ""),
            "source_file": row.get("source_file", ""),
            # case 관련 (있으면 추가)
            "case_number": row.get("case_number", ""),
            "keywords": row.get("keywords", "")
        }
        
        # 빈 값 제거
        metadata = {k: v for k, v in metadata.items() if v}
        
        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)
    
    print(f"\n✅ {len(documents)}개 Document 생성 완료")
    return documents

# 실행
documents = csv_to_documents(df)

Document 변환 중: 100%|██████████| 190/190 [00:00<00:00, 12018.79it/s]


✅ 190개 Document 생성 완료


In [13]:
# Document 샘플 확인
print("📄 Document 샘플:")
print(f"Content: {documents[0].page_content[:100]}...")
print(f"\nMetadata: {documents[0].metadata}")

📄 Document 샘플:
Content: 채무자가 채무의 내용에 좇은 이행을 하지 아니한 때에는 채권자는 손해배상을 청구할 수 있다. 그러나 채무자의 고의나 과실없이 이행할 수 없게 된 때에는 그러하지 아니하다. 제2장 ...

Metadata: {'id': '2530eb03-cc6c-4ddf-8747-7d204c1832ac', 'law_family': '주택임대차', 'law_name': '민법', 'law_type': '일반법', 'article': '제390조', 'article_title': '채무불이행과 손해배상', 'priority': 4, 'effective_date': '2026-01-02', 'source_file': '민법(법률)(제20432호)(20260101)-간략.docx'}


In [14]:
##  5단계: Upstage 임베딩 모델 초기화

In [15]:
print("📦 Upstage 임베딩 모델 로드 중...")

embedding = UpstageEmbeddings(
    model="solar-embedding-1-large-passage",
    api_key=UPSTAGE_API_KEY
)

print("✅ 임베딩 모델 준비 완료")

📦 Upstage 임베딩 모델 로드 중...
✅ 임베딩 모델 준비 완료


In [16]:
# 임베딩 테스트
test_text = "보증금 반환을 안 해주면 어떻게 하나요?"
test_vector = embedding.embed_query(test_text)

print(f"테스트 문장: {test_text}")
print(f"벡터 차원: {len(test_vector)}")
print(f"벡터 샘플: {test_vector[:5]}")

테스트 문장: 보증금 반환을 안 해주면 어떻게 하나요?
벡터 차원: 4096
벡터 샘플: [0.023345947265625, -0.022552490234375, 0.014373779296875, 0.0017900466918945312, -0.00876617431640625]


In [19]:
## 6단계: Pinecone에 업로드

In [20]:
print(f"📤 Pinecone 업로드 시작...")
print(f"   - 인덱스: {INDEX_NAME}")
print(f"   - 문서 수: {len(documents)}")

# PineconeVectorStore로 업로드
vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=embedding,
    index_name=INDEX_NAME
)

print("✅ Pinecone 업로드 완료!")

📤 Pinecone 업로드 시작...
   - 인덱스: real-estate-law-index
   - 문서 수: 190
✅ Pinecone 업로드 완료!


In [21]:
##  7단계: 업로드 테스트 - 검색 수행

In [22]:
# 테스트 검색
test_queries = [
    "보증금을 못 받으면 어떻게 하나요?",
    "임대인의 수선의무",
    "대항력 취득 요건"
]

for query in test_queries:
    print(f"\n{'='*70}")
    print(f"🔍 질의: {query}")
    print(f"{'-'*70}")
    
    # 검색
    results = vectorstore.similarity_search(query, k=3)
    
    for i, doc in enumerate(results, 1):
        meta = doc.metadata
        print(f"\n[{i}] 📌 Priority {meta.get('priority', '?')}")
        print(f"    📄 {meta.get('law_name', 'Unknown')} - {meta.get('article', 'Unknown')}")
        print(f"    📝 {doc.page_content[:100]}...")


🔍 질의: 보증금을 못 받으면 어떻게 하나요?
----------------------------------------------------------------------

[1] 📌 Priority 7.0
    📄 상담사례집 - 사례1
    📝 대 지원은 없을까요?
12.   신탁사기 피해주택 거주 지원 36 
이사 온 지 얼마 되지 않아 신탁사로부터 공매 예정이니 퇴거해야 한다는 통지를 받았습니다.  피해주택에 계속 거...

[2] 📌 Priority 7.0
    📄 상담사례집 - 사례1
    📝 사례 
1.   전세대출 연체 문제  27 
전세피해주택에서 퇴거하고자 하나 임대인이 연락 두절(또는 구속, 보증금 반환불가 통지 등) 되었습니다.  
곧 전세대출 만기인데 대출이...

[3] 📌 Priority 7.0
    📄 상담사례집 - 사례40
    📝 사례
40 4119 임대인의 사망 문제
임대인이 사망하였는데 누구에게 보증금을 돌려받아야 하나요?
l   임대인이 사망하였다면 피해주택 상속인에게 보증금 반환을 청구할 수 있고 상...

🔍 질의: 임대인의 수선의무
----------------------------------------------------------------------

[1] 📌 Priority 4.0
    📄 민법 - 제634조
    📝 임차물의 수리를 요하거나 임차물에 대하여 권리를 주장하는 자가 있는 때에는 임차인은 지체없이 임대인에게 이를 통지하여야 한다. 그러나 임대인이 이미 이를 안 때에는 그러하지 아니하...

[2] 📌 Priority 4.0
    📄 민법 - 제623조
    📝 임대인은 목적물을 임차인에게 인도하고 계약존속중 그 사용, 수익에 필요한 상태를 유지하게 할 의무를 부담한다....

[3] 📌 Priority 4.0
    📄 민법 - 제624조
    📝 임대인이 임대물의 보존에 필요한 행위를 하는 때에는 임차인은 이를 거절하지 못한다....

🔍 질의: 대항력 취득 요건
----

In [23]:
## 9단계: Pinecone 인덱스 통계 확인

In [24]:
# 인덱스 정보
index = pc.Index(INDEX_NAME)
stats = index.describe_index_stats()

print("📊 Pinecone 인덱스 통계:")
print(f"   - Dimension: {stats['dimension']}")
print(f"   - Total Vectors: {stats['total_vector_count']}")
print(f"   - Index Fullness: {stats.get('index_fullness', 'N/A')}")

📊 Pinecone 인덱스 통계:
   - Dimension: 4096
   - Total Vectors: 190
   - Index Fullness: 0.0
